# Training Data Preparation

In [6]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Set GPU configuration to avoid memory issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], 
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [8]:
# Constants
IMG_SIZE = (299, 299)  # Target image size for InceptionV3

In [9]:
def load_images_from_directory(directory, label):
    """
    Load and preprocess images from the specified directory.
    
    Parameters:
    - directory (str): Path to the directory containing images.
    - label (int): Label for the samples (0 for real, 1 for fake).
    
    Returns:
    - data (np.array): Array of processed images.
    - labels (np.array): Array of corresponding labels.
    """
    data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(('.jpeg', '.jpg', '.png')):
            filepath = os.path.join(directory, filename)
            img = cv2.imread(filepath)
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)
                img = img_to_array(img)
                img = preprocess_input(img)
                data.append(img)
                labels.append(label)
    
    return np.array(data), np.array(labels)

In [10]:
# Load real and fake images
REAL_DIR = "./DFFD/real"
FAKE_DIR = "./DFFD/fake"

x_real, y_real = load_images_from_directory(REAL_DIR, label=0)  # Label 0 for real
x_fake, y_fake = load_images_from_directory(FAKE_DIR, label=1)  # Label 1 for fake

In [11]:
# Combine real and fake data
x_data = np.concatenate([x_real, x_fake], axis=0)
y_data = np.concatenate([y_real, y_fake], axis=0)

In [12]:
# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [13]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(x_train)

In [14]:
# Define the feature extractor using InceptionV3
def create_feature_extractor():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=x)
    return feature_extractor

In [15]:
# Build the model
def create_model():
    feature_extractor = create_feature_extractor()
    
    model = Sequential()
    model.add(Input(shape=(299, 299, 3)))  # Input shape for single images
    model.add(feature_extractor)  # Feature extraction
    model.add(Dense(64, activation='relu'))  # Dense layer for intermediate features
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))  # Output layer: binary classification
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# Create and summarize the model
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 21,933,985
Trainable params: 21,899,553
Non-trainable params: 34,432
_________________________________________________________________


In [17]:
# Learning rate scheduler
lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                  patience=3, 
                                  verbose=1, 
                                  factor=0.5, 
                                  min_lr=1e-6)

In [19]:
# Train the model
model.fit(datagen.flow(x_train, y_train, batch_size=16), 
          epochs=10, 
          validation_data=(x_val, y_val), 
          callbacks=[lr_reduction])

Epoch 1/10
107/107 [==============================] - 56s 451ms/step - loss: 0.5531 - accuracy: 0.7494 - val_loss: 15.0733 - val_accuracy: 0.3341 - lr: 0.0010
Epoch 2/10
107/107 [==============================] - 41s 384ms/step - loss: 0.5388 - accuracy: 0.7488 - val_loss: 620.3170 - val_accuracy: 0.3598 - lr: 0.0010
Epoch 3/10
107/107 [==============================] - 42s 388ms/step - loss: 0.5279 - accuracy: 0.7717 - val_loss: 3.7357 - val_accuracy: 0.6449 - lr: 0.0010
Epoch 4/10
107/107 [==============================] - 42s 391ms/step - loss: 0.5205 - accuracy: 0.7740 - val_loss: 0.9973 - val_accuracy: 0.6192 - lr: 0.0010
Epoch 5/10
107/107 [==============================] - 42s 393ms/step - loss: 0.5688 - accuracy: 0.7196 - val_loss: 4398.3774 - val_accuracy: 0.3341 - lr: 0.0010
Epoch 6/10
107/107 [==============================] - 43s 398ms/step - loss: 0.5774 - accuracy: 0.7149 - val_loss: 30.7413 - val_accuracy: 0.4579 - lr: 0.0010
Epoch 7/10
107/107 [=========================

In [20]:
# Save the final model
model.save('real2_76%.h5')

In [33]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Load the saved model
model = tf.keras.models.load_model('real2_76%.h5')

# Constants
IMG_SIZE = (299, 299)  # Target image size for InceptionV3

def preprocess_image(image_path):
    """
    Load and preprocess an image for prediction.
    
    Parameters:
    - image_path (str): Path to the image file.
    
    Returns:
    - img (np.array): Preprocessed image array.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or cannot be loaded.")
    
    # Resize and preprocess the image
    img = cv2.resize(img, IMG_SIZE)
    img = img_to_array(img)
    img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image_class(image_path):
    """
    Predict the class of an image using the loaded model.
    
    Parameters:
    - image_path (str): Path to the image file.
    
    Returns:
    - class_label (int): Predicted class label (0 for real, 1 for fake).
    """
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    class_label = (prediction > 0.5).astype(int)  # Convert prediction to class label
    return class_label[0][0]

# Example usage
image_path = 'DFFD/real/real_2_198.png'  # Replace with your image path
predicted_class = predict_image_class(image_path)

if predicted_class == 0:
    print("The image is classified as REAL.")
else:
    print("The image is classified as FAKE.")

1/1 [==============================] - 1s 764ms/step
The image is classified as REAL.
